##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX 에스티 메이터 컴포넌트 튜토리얼

***TensorFlow Extended (TFX)에 대한 구성 요소 별 소개***

참고 : 설정이 필요하지 않은 Colab 노트북에서이 가이드를 실행하는 것이 좋습니다. "Google Colab에서 실행"을 클릭하십시오.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/components"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서보기</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/components.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/components.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스보기</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tfx/tutorials/tfx/components.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table></div>

> 경고: Estimator는 새 코드에 권장되지 않습니다. Estimator는 `v1.Session` 스타일 코드를 실행하며, 이 코드는 올바르게 작성하기가 좀 더 어렵고 특히 TF 2 코드와 결합할 경우 예기치 않게 작동할 수 있습니다. Estimator는 [호환성 보장](https://tensorflow.org/guide/versions)이 적용되지만 보안 취약점 외에는 수정 사항이 제공되지 않습니다. 자세한 내용은 [마이그레이션 가이드](https://tensorflow.org/guide/migrate)를 참조하세요.

이 Colab 기반 자습서는 TensorFlow Extended (TFX)의 각 기본 제공 구성 요소를 대화식으로 안내합니다.

데이터 수집에서 모델 푸시, 제공에 이르기까지 엔드 투 엔드 머신 러닝 파이프 라인의 모든 단계를 다룹니다.

완료되면이 노트북의 콘텐츠를 TFX 파이프 라인 소스 코드로 자동으로 내보낼 수 있으며 Apache Airflow 및 Apache Beam으로 조정할 수 있습니다.

참고 :이 노트북 및 관련 API는 **실험적** 이며 현재 개발 중입니다. 기능, 동작 및 표현의 주요 변경이 예상됩니다.

## 배경 설명

이 노트북은 Jupyter / Colab 환경에서 TFX를 사용하는 방법을 보여줍니다. 여기에서는 대화 형 노트북에서 Chicago Taxi 예제를 살펴 봅니다.

대화 형 노트북에서 작업하는 것은 TFX 파이프 라인의 구조에 익숙해지는 데 유용한 방법입니다. 경량 개발 환경으로 자체 파이프 라인을 개발할 때도 유용하지만 대화 형 노트북이 오케스트레이션되는 방식과 메타 데이터 아티팩트에 액세스하는 방식에 차이가 있다는 점을 알고 있어야합니다.

### 관현악법

TFX의 프로덕션 배포에서는 Apache Airflow, Kubeflow Pipelines 또는 Apache Beam과 같은 오케 스트레이터를 사용하여 TFX 구성 요소의 사전 정의 된 파이프 라인 그래프를 오케스트레이션합니다. 대화 형 노트북에서 노트북 자체는 노트북 셀을 실행할 때 각 TFX 구성 요소를 실행하는 오케 스트레이터입니다.

### 메타 데이터

TFX의 프로덕션 배포에서 ML Metadata (MLMD) API를 통해 메타 데이터에 액세스합니다. MLMD는 MySQL 또는 SQLite와 같은 데이터베이스에 메타 데이터 속성을 저장하고 파일 시스템과 같은 영구 저장소에 메타 데이터 페이로드를 저장합니다. 대화 형 노트북에서 속성과 페이로드는 모두 Jupyter 노트북 또는 Colab 서버 `/tmp`

## Setup

먼저 필요한 패키지를 설치 및 가져오고 경로를 설정하고 데이터를 다운로드합니다.

### Pip 업그레이드

로컬에서 실행할 때 시스템에서 Pip을 업그레이드하지 않으려면 Colab에서 실행 중인지 확인하십시오. 물론 로컬 시스템은 별도로 업그레이드 할 수 있습니다.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### TFX 설치

**참고 : Google Colab에서는 패키지 업데이트로 인해이 셀을 처음 실행할 때 런타임을 다시 시작해야합니다 (런타임&gt; 런타임 다시 시작 ...).**

In [ ]:
!pip install -U tfx

## 런타임을 다시 시작 했습니까?

Google Colab을 사용하는 경우 위의 셀을 처음 실행할 때 런타임을 다시 시작해야합니다 (런타임&gt; 런타임 다시 시작 ...). 이는 Colab이 패키지를로드하는 방식 때문입니다.

### 패키지 가져오기

표준 TFX 컴포넌트 클래스를 포함하여 필요한 패키지를 가져옵니다.

In [ ]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

라이브러리 버전을 확인해 봅시다.

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

### 파이프 라인 경로 설정

In [ ]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Chicago Taxi Pipeline example.
_taxi_root = os.path.join(_tfx_root, 'examples/chicago_taxi_pipeline')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/taxi_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

### 예제 데이터 다운로드

TFX 파이프 라인에서 사용할 예제 데이터 세트를 다운로드합니다.

우리가 사용하는 데이터 세트는 시카고시에서 발표 한 [Taxi Trips 데이터 세트입니다.](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) 이 데이터 세트의 열은 다음과 같습니다.

<table>
<tr>
<td>pickup_community_area</td>
<td>요금</td>
<td>trip_start_month</td>
</tr>
<tr>
<td>trip_start_hour</td>
<td>trip_start_day</td>
<td>trip_start_timestamp</td>
</tr>
<tr>
<td>픽업 _ 위도</td>
<td>픽업 _ 경도</td>
<td>dropoff_latitude</td>
</tr>
<tr>
<td>dropoff_longitude</td>
<td>trip_miles</td>
<td>Pickup_census_tract</td>
</tr>
<tr>
<td>dropoff_census_tract</td>
<td>지불 _ 유형</td>
<td>회사</td>
</tr>
<tr>
<td>trip_seconds</td>
<td>dropoff_community_area</td>
<td>팁</td>
</tr>
</table>

이 데이터 세트를 사용하여 여행 `tips` 을 예측하는 모델을 구축합니다.

In [ ]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

CSV 파일을 빠르게 살펴보십시오.

In [ ]:
!head {_data_filepath}

*면책 조항 :이 사이트는 시카고시의 공식 웹 사이트 인 www.cityofchicago.org의 원본 소스에서 사용하기 위해 수정 된 데이터를 사용하는 애플리케이션을 제공합니다. 시카고시는이 사이트에서 제공되는 데이터의 내용, 정확성, 적시성 또는 완전성에 대해 어떠한 주장도하지 않습니다. 이 사이트에서 제공되는 데이터는 언제든지 변경 될 수 있습니다. 이 사이트에서 제공하는 데이터는 자신의 책임하에 사용되는 것으로 이해됩니다.*

### InteractiveContext 만들기

마지막으로이 노트북에서 TFX 구성 요소를 대화식으로 실행할 수있는 InteractiveContext를 만듭니다.

In [ ]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

## 대화식으로 TFX 구성 요소 실행

다음 셀에서는 TFX 구성 요소를 하나씩 만들고 각 구성 요소를 실행하고 출력 아티팩트를 시각화합니다.

### ExampleGen

`ExampleGen` 구성 요소는 일반적으로 TFX 파이프 라인의 시작 부분에 있습니다. 그것은 :

1. 데이터를 훈련 및 평가 세트로 분할 (기본적으로 2/3 훈련 + 1/3 평가)
2. 데이터를 `tf.Example` 형식으로 변환합니다([여기](https://www.tensorflow.org/tutorials/load_data/tfrecord)에서 자세히 알아보기).
3. 다른 구성 요소가 액세스 할 수 있도록 `_tfx_root` 디렉토리에 복사하십시오.

`ExampleGen` 은 데이터 소스에 대한 경로를 입력으로 사용합니다. 이 경우 다운로드 된 CSV가 포함 된 `_data_root`

참고 :이 노트북에서는 구성 요소를 하나씩 인스턴스화하고 `InteractiveContext.run()` 실행할 수 있습니다. `Pipeline` 모든 구성 요소를 사전에 지정하여 오케 스트레이터에 전달할 것입니다 ( [TFX 파이프 라인 구축 가이드 참조](https://www.tensorflow.org/tfx/guide/build_tfx_pipeline) ).

In [ ]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
context.run(example_gen)

`ExampleGen` 의 출력 결과물을 살펴 보겠습니다. 이 구성 요소는 두 가지 아티팩트, 교육 예제 및 평가 예제를 생성합니다.

In [ ]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

처음 세 가지 교육 예제도 살펴볼 수 있습니다.

In [ ]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

`ExampleGen` 이 데이터 수집을 완료 했으므로 이제 다음 단계는 데이터 분석입니다.

### StatisticsGen

`StatisticsGen` 구성 요소는 데이터 분석 및 다운 스트림 구성 요소에서 사용하기 위해 데이터 세트에 대한 통계를 계산합니다. [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) 라이브러리를 사용합니다.

`StatisticsGen` `ExampleGen` 사용하여 방금 수집 한 데이터 세트를 입력으로받습니다.

In [ ]:
statistics_gen = tfx.components.StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

`StatisticsGen` 실행이 끝나면 출력 된 통계를 시각화 할 수 있습니다. 다른 플롯으로 놀아보세요!

In [ ]:
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

`SchemaGen` 구성 요소는 데이터 통계를 기반으로 스키마를 생성합니다. (스키마는 데이터 세트에있는 기능의 예상 경계, 유형 및 속성을 정의합니다.) 또한 [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) 라이브러리를 사용합니다.

`SchemaGen` `StatisticsGen` 생성 한 통계를 입력으로 받아 기본적으로 훈련 분할을 살펴 봅니다.

In [ ]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

`SchemaGen` 실행이 끝나면 생성 된 스키마를 테이블로 시각화 할 수 있습니다.

In [ ]:
context.show(schema_gen.outputs['schema'])

데이터 세트의 각 특성은 속성과 함께 스키마 테이블에 행으로 표시됩니다. 또한 스키마는 도메인으로 표시되는 범주 형 기능이 취하는 모든 값을 캡처합니다.

스키마에 대한 자세한 내용 [은 SchemaGen 문서를](https://www.tensorflow.org/tfx/guide/schemagen) 참조하십시오.

### ExampleValidator

`ExampleValidator` 구성 요소는 스키마에 정의 된 기대치를 기반으로 데이터의 이상을 감지합니다. [또한 TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) 라이브러리를 사용합니다.

`ExampleValidator` 는 `StatisticsGen` `SchemaGen` 의 스키마를 입력으로받습니다.

In [ ]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

`ExampleValidator` 가 실행을 마치면 이상을 테이블로 시각화 할 수 있습니다.

In [ ]:
context.show(example_validator.outputs['anomalies'])

anomalies 테이블에서 이상이 없음을 확인할 수 있습니다. 이것이 우리가 분석 한 첫 번째 데이터 세트이고 스키마가 이에 맞게 조정 되었기 때문에 이것이 우리가 기대하는 것입니다. 이 스키마를 검토해야합니다. 예상치 못한 것은 데이터의 이상을 의미합니다. 검토가 완료되면 스키마를 사용하여 향후 데이터를 보호 할 수 있으며 여기에서 생성 된 이상 항목을 사용하여 모델 성능을 디버깅하고 시간이 지남에 따라 데이터가 어떻게 진화하는지 이해하며 데이터 오류를 식별 할 수 있습니다.

### 변환

`Transform` 구성 요소는 학습 및 제공 모두를위한 기능 엔지니어링을 수행합니다. [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started) 라이브러리를 사용합니다.

`Transform` `ExampleGen` 의 데이터 `SchemaGen` 의 스키마 및 사용자 정의 Transform 코드를 포함하는 모듈을 입력으로받습니다.

아래에서 사용자 정의 Transform 코드의 예를 살펴 보겠습니다 (TensorFlow Transform API에 대한 소개 [는 자습서 참조](https://www.tensorflow.org/tfx/tutorials/transform/simple) ). 먼저 기능 엔지니어링을위한 몇 가지 상수를 정의합니다.

참고 : `%%writefile` 셀 매직은 셀의 내용을 디스크에 `.py` 이렇게하면 `Transform` 구성 요소가 코드를 모듈로로드 할 수 있습니다.


In [ ]:
_taxi_constants_module_file = 'taxi_constants.py'

In [ ]:
%%writefile {_taxi_constants_module_file}

# Categorical features are assumed to each have a maximum value in the dataset.
MAX_CATEGORICAL_FEATURE_VALUES = [24, 31, 12]

CATEGORICAL_FEATURE_KEYS = [
    'trip_start_hour', 'trip_start_day', 'trip_start_month',
    'pickup_census_tract', 'dropoff_census_tract', 'pickup_community_area',
    'dropoff_community_area'
]

DENSE_FLOAT_FEATURE_KEYS = ['trip_miles', 'fare', 'trip_seconds']

# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = 10

BUCKET_FEATURE_KEYS = [
    'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
    'dropoff_longitude'
]

# Number of vocabulary terms used for encoding VOCAB_FEATURES by tf.transform
VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized VOCAB_FEATURES are hashed.
OOV_SIZE = 10

VOCAB_FEATURE_KEYS = [
    'payment_type',
    'company',
]

# Keys
LABEL_KEY = 'tips'
FARE_KEY = 'fare'

다음으로 원시 데이터를 입력으로 받아들이고 모델이 학습 할 수있는 변환 된 기능을 반환 `preprocessing_fn` 을 작성합니다.

In [ ]:
_taxi_transform_module_file = 'taxi_transform.py'

In [ ]:
%%writefile {_taxi_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import taxi_constants

_DENSE_FLOAT_FEATURE_KEYS = taxi_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = taxi_constants.VOCAB_FEATURE_KEYS
_VOCAB_SIZE = taxi_constants.VOCAB_SIZE
_OOV_SIZE = taxi_constants.OOV_SIZE
_FEATURE_BUCKET_COUNT = taxi_constants.FEATURE_BUCKET_COUNT
_BUCKET_FEATURE_KEYS = taxi_constants.BUCKET_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = taxi_constants.CATEGORICAL_FEATURE_KEYS
_FARE_KEY = taxi_constants.FARE_KEY
_LABEL_KEY = taxi_constants.LABEL_KEY


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _DENSE_FLOAT_FEATURE_KEYS:
    # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
    outputs[key] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]))

  for key in _VOCAB_FEATURE_KEYS:
    # Build a vocabulary for this feature.
    outputs[key] = tft.compute_and_apply_vocabulary(
        _fill_in_missing(inputs[key]),
        top_k=_VOCAB_SIZE,
        num_oov_buckets=_OOV_SIZE)

  for key in _BUCKET_FEATURE_KEYS:
    outputs[key] = tft.bucketize(
        _fill_in_missing(inputs[key]), _FEATURE_BUCKET_COUNT)

  for key in _CATEGORICAL_FEATURE_KEYS:
    outputs[key] = _fill_in_missing(inputs[key])

  # Was this passenger a big tipper?
  taxi_fare = _fill_in_missing(inputs[_FARE_KEY])
  tips = _fill_in_missing(inputs[_LABEL_KEY])
  outputs[_LABEL_KEY] = tf.where(
      tf.math.is_nan(taxi_fare),
      tf.cast(tf.zeros_like(taxi_fare), tf.int64),
      # Test if the tip was > 20% of the fare.
      tf.cast(
          tf.greater(tips, tf.multiply(taxi_fare, tf.constant(0.2))), tf.int64))

  return outputs


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  if not isinstance(x, tf.sparse.SparseTensor):
    return x

  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

이제이 기능 엔지니어링 코드를 `Transform` 구성 요소에 전달하고이를 실행하여 데이터를 변환합니다.

In [ ]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_taxi_transform_module_file))
context.run(transform)

`Transform` 의 출력 결과물을 살펴 보겠습니다. 이 구성 요소는 두 가지 유형의 출력을 생성합니다.

- `transform_graph` 는 전처리 작업을 수행 할 수있는 그래프입니다 (이 그래프는 제공 및 평가 모델에 포함됨).
- `transformed_examples` 는 전처리 된 훈련 및 평가 데이터를 나타냅니다.

In [ ]:
transform.outputs

`transform_graph` 아티팩트를 살펴보십시오. 세 개의 하위 디렉터리를 포함하는 디렉터리를 가리 킵니다.

In [ ]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

`transformed_metadata` 하위 디렉토리에는 전처리 된 데이터의 스키마가 포함됩니다. `transform_fn` 하위 디렉토리에는 실제 전처리 그래프가 포함되어 있습니다. `metadata` 하위 디렉토리에는 원본 데이터의 스키마가 포함됩니다.

처음 세 가지 변형 된 예를 살펴볼 수도 있습니다.

In [ ]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

`Transform` 구성 요소가 데이터를 특성으로 변환 한 후 다음 단계는 모델을 학습하는 것입니다.

### Trainer

`Trainer` 구성 요소는 TensorFlow에서 정의한 모델을 학습시킵니다 (Estimator API 또는 [`model_to_estimator`](https://www.tensorflow.org/api_docs/python/tf/keras/estimator/model_to_estimator) ).

`Trainer` `SchemaGen` 의 스키마 `Transform` 의 변환 된 데이터 및 그래프, 학습 매개 변수, 사용자 정의 모델 코드가 포함 된 모듈을 입력으로받습니다.

아래에서 사용자 정의 모델 코드의 예를 살펴 보겠습니다 (TensorFlow Estimator API에 대한 소개 [는 튜토리얼 참조](https://www.tensorflow.org/tutorials/estimator/premade) ).

In [ ]:
_taxi_trainer_module_file = 'taxi_trainer.py'

In [ ]:
%%writefile {_taxi_trainer_module_file}

import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils
from tfx_bsl.tfxio import dataset_options

import taxi_constants

_DENSE_FLOAT_FEATURE_KEYS = taxi_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = taxi_constants.VOCAB_FEATURE_KEYS
_VOCAB_SIZE = taxi_constants.VOCAB_SIZE
_OOV_SIZE = taxi_constants.OOV_SIZE
_FEATURE_BUCKET_COUNT = taxi_constants.FEATURE_BUCKET_COUNT
_BUCKET_FEATURE_KEYS = taxi_constants.BUCKET_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = taxi_constants.CATEGORICAL_FEATURE_KEYS
_MAX_CATEGORICAL_FEATURE_VALUES = taxi_constants.MAX_CATEGORICAL_FEATURE_VALUES
_LABEL_KEY = taxi_constants.LABEL_KEY


# Tf.Transform considers these features as "raw"
def _get_raw_feature_spec(schema):
  return schema_utils.schema_as_feature_spec(schema).feature_spec


def _build_estimator(config, hidden_units=None, warm_start_from=None):
  """Build an estimator for predicting the tipping behavior of taxi riders.
  Args:
    config: tf.estimator.RunConfig defining the runtime environment for the
      estimator (including model_dir).
    hidden_units: [int], the layer sizes of the DNN (input layer first)
    warm_start_from: Optional directory to warm start from.
  Returns:
    A dict of the following:
      - estimator: The estimator that will be used for training and eval.
      - train_spec: Spec for training.
      - eval_spec: Spec for eval.
      - eval_input_receiver_fn: Input function for eval.
  """
  real_valued_columns = [
      tf.feature_column.numeric_column(key, shape=())
      for key in _DENSE_FLOAT_FEATURE_KEYS
  ]
  categorical_columns = [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=_VOCAB_SIZE + _OOV_SIZE, default_value=0)
      for key in _VOCAB_FEATURE_KEYS
  ]
  categorical_columns += [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=_FEATURE_BUCKET_COUNT, default_value=0)
      for key in _BUCKET_FEATURE_KEYS
  ]
  categorical_columns += [
      tf.feature_column.categorical_column_with_identity(  # pylint: disable=g-complex-comprehension
          key,
          num_buckets=num_buckets,
          default_value=0) for key, num_buckets in zip(
              _CATEGORICAL_FEATURE_KEYS,
              _MAX_CATEGORICAL_FEATURE_VALUES)
  ]
  return tf.estimator.DNNLinearCombinedClassifier(
      config=config,
      linear_feature_columns=categorical_columns,
      dnn_feature_columns=real_valued_columns,
      dnn_hidden_units=hidden_units or [100, 70, 50, 25],
      warm_start_from=warm_start_from)


def _example_serving_receiver_fn(tf_transform_graph, schema):
  """Build the serving in inputs.
  Args:
    tf_transform_graph: A TFTransformOutput.
    schema: the schema of the input data.
  Returns:
    Tensorflow graph which parses examples, applying tf-transform to them.
  """
  raw_feature_spec = _get_raw_feature_spec(schema)
  raw_feature_spec.pop(_LABEL_KEY)

  raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
      raw_feature_spec, default_batch_size=None)
  serving_input_receiver = raw_input_fn()

  transformed_features = tf_transform_graph.transform_raw_features(
      serving_input_receiver.features)

  return tf.estimator.export.ServingInputReceiver(
      transformed_features, serving_input_receiver.receiver_tensors)


def _eval_input_receiver_fn(tf_transform_graph, schema):
  """Build everything needed for the tf-model-analysis to run the model.
  Args:
    tf_transform_graph: A TFTransformOutput.
    schema: the schema of the input data.
  Returns:
    EvalInputReceiver function, which contains:
      - Tensorflow graph which parses raw untransformed features, applies the
        tf-transform preprocessing operators.
      - Set of raw, untransformed features.
      - Label against which predictions will be compared.
  """
  # Notice that the inputs are raw features, not transformed features here.
  raw_feature_spec = _get_raw_feature_spec(schema)

  serialized_tf_example = tf.compat.v1.placeholder(
      dtype=tf.string, shape=[None], name='input_example_tensor')

  # Add a parse_example operator to the tensorflow graph, which will parse
  # raw, untransformed, tf examples.
  features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)

  # Now that we have our raw examples, process them through the tf-transform
  # function computed during the preprocessing step.
  transformed_features = tf_transform_graph.transform_raw_features(
      features)

  # The key name MUST be 'examples'.
  receiver_tensors = {'examples': serialized_tf_example}

  # NOTE: Model is driven by transformed features (since training works on the
  # materialized output of TFT, but slicing will happen on raw features.
  features.update(transformed_features)

  return tfma.export.EvalInputReceiver(
      features=features,
      receiver_tensors=receiver_tensors,
      labels=transformed_features[_LABEL_KEY])


def _input_fn(file_pattern, data_accessor, tf_transform_output, batch_size=200):
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      tf_transform_output.transformed_metadata.schema)


# TFX will call this function
def trainer_fn(trainer_fn_args, schema):
  """Build the estimator using the high level API.
  Args:
    trainer_fn_args: Holds args used to train the model as name/value pairs.
    schema: Holds the schema of the training examples.
  Returns:
    A dict of the following:
      - estimator: The estimator that will be used for training and eval.
      - train_spec: Spec for training.
      - eval_spec: Spec for eval.
      - eval_input_receiver_fn: Input function for eval.
  """
  # Number of nodes in the first layer of the DNN
  first_dnn_layer_size = 100
  num_dnn_layers = 4
  dnn_decay_factor = 0.7

  train_batch_size = 40
  eval_batch_size = 40

  tf_transform_graph = tft.TFTransformOutput(trainer_fn_args.transform_output)

  train_input_fn = lambda: _input_fn(  # pylint: disable=g-long-lambda
      trainer_fn_args.train_files,
      trainer_fn_args.data_accessor,
      tf_transform_graph,
      batch_size=train_batch_size)

  eval_input_fn = lambda: _input_fn(  # pylint: disable=g-long-lambda
      trainer_fn_args.eval_files,
      trainer_fn_args.data_accessor,
      tf_transform_graph,
      batch_size=eval_batch_size)

  train_spec = tf.estimator.TrainSpec(  # pylint: disable=g-long-lambda
      train_input_fn,
      max_steps=trainer_fn_args.train_steps)

  serving_receiver_fn = lambda: _example_serving_receiver_fn(  # pylint: disable=g-long-lambda
      tf_transform_graph, schema)

  exporter = tf.estimator.FinalExporter('chicago-taxi', serving_receiver_fn)
  eval_spec = tf.estimator.EvalSpec(
      eval_input_fn,
      steps=trainer_fn_args.eval_steps,
      exporters=[exporter],
      name='chicago-taxi-eval')

  run_config = tf.estimator.RunConfig(
      save_checkpoints_steps=999, keep_checkpoint_max=1)

  run_config = run_config.replace(model_dir=trainer_fn_args.serving_model_dir)

  estimator = _build_estimator(
      # Construct layers sizes with exponetial decay
      hidden_units=[
          max(2, int(first_dnn_layer_size * dnn_decay_factor**i))
          for i in range(num_dnn_layers)
      ],
      config=run_config,
      warm_start_from=trainer_fn_args.base_model)

  # Create an input receiver for TFMA processing
  receiver_fn = lambda: _eval_input_receiver_fn(  # pylint: disable=g-long-lambda
      tf_transform_graph, schema)

  return {
      'estimator': estimator,
      'train_spec': train_spec,
      'eval_spec': eval_spec,
      'eval_input_receiver_fn': receiver_fn
  }

이제이 모델 코드를 `Trainer` 구성 요소에 전달하고 실행하여 모델을 교육합니다.

In [ ]:
from tfx.components.trainer.executor import Executor
from tfx.dsl.components.base import executor_spec

trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_taxi_trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(Executor),
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=tfx.proto.TrainArgs(num_steps=10000),
    eval_args=tfx.proto.EvalArgs(num_steps=5000))
context.run(trainer)

#### TensorBoard로 훈련 분석

선택적으로 TensorBoard를 Trainer에 연결하여 모델의 훈련 곡선을 분석 할 수 있습니다.

In [ ]:
# Get the URI of the output artifact representing the training logs, which is a directory
model_run_dir = trainer.outputs['model_run'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_dir}

### 평가자

`Evaluator` 구성 요소는 평가 세트에 대한 모델 성능 메트릭을 계산합니다. [TensorFlow Model Analysis](https://www.tensorflow.org/tfx/model_analysis/get_started) 라이브러리를 사용합니다. 또한 `Evaluator` 는 새로 훈련 된 모델이 이전 모델보다 나은지 선택적으로 검증 할 수도 있습니다. 이는 매일 모델을 자동으로 학습하고 검증 할 수있는 프로덕션 파이프 라인 설정에서 유용합니다. 이 노트북에서는 하나의 모델 만 학습하므로 `Evaluator` 자동으로 모델을 "양호"로 표시합니다.

`Evaluator` `ExampleGen` `Trainer` 의 학습 된 모델 및 슬라이싱 구성의 데이터를 입력으로받습니다. 슬라이싱 구성을 사용하면 특성 값에 대한 메트릭을 분할 할 수 있습니다 (예 : 오전 8시 대 오후 8시에 시작하는 택시 여행에서 모델의 성능이 어떻습니까?). 아래에서이 구성의 예를 참조하십시오.

In [ ]:
eval_config = tfma.EvalConfig(
    model_specs=[
        # Using signature 'eval' implies the use of an EvalSavedModel. To use
        # a serving model remove the signature to defaults to 'serving_default'
        # and add a label_key.
        tfma.ModelSpec(signature_name='eval')
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount')
            ],
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            thresholds = {
                'accuracy': tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.5}),
                    # Change threshold will be ignored if there is no
                    # baseline model resolved from MLMD (first run).
                    change_threshold=tfma.GenericChangeThreshold(
                       direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                       absolute={'value': -1e-10}))
            }
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
        tfma.SlicingSpec(feature_keys=['trip_start_hour'])
    ])

다음으로이 구성을 `Evaluator` 하고 실행합니다.

In [ ]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')
context.run(model_resolver)

evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    eval_config=eval_config)
context.run(evaluator)

`Evaluator` 의 출력 결과물을 살펴 보겠습니다. 

In [ ]:
evaluator.outputs

`evaluation` 출력을 사용하여 전체 평가 세트에 대한 글로벌 메트릭의 기본 시각화를 표시 할 수 있습니다.

In [ ]:
context.show(evaluator.outputs['evaluation'])

슬라이스 된 평가 측정 항목의 시각화를 보려면 TensorFlow 모델 분석 라이브러리를 직접 호출 할 수 있습니다.

In [ ]:
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='trip_start_hour')

이 시각화는 동일한 측정 항목을 보여 주지만 전체 평가 세트 대신 `trip_start_hour`

TensorFlow 모델 분석은 공정성 지표 및 모델 성능의 시계열 플로팅과 같은 다른 많은 시각화를 지원합니다. 자세한 내용 [은 튜토리얼을](https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic) 참조하십시오.

구성에 임계 값을 추가 했으므로 유효성 검사 출력도 사용할 수 있습니다. `blessing` 아티팩트의 우선 순위는 모델이 유효성 검사를 통과했음을 나타냅니다. 이것이 수행되는 첫 번째 검증이기 때문에 후보자는 자동으로 축복을받습니다.

In [ ]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

이제 유효성 검사 결과 레코드를로드하여 성공 여부를 확인할 수도 있습니다.

In [ ]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

### 미는 사람

`Pusher` 구성 요소는 일반적으로 TFX 파이프 라인의 끝에 있습니다. 모델이 유효성 검사를 통과했는지 확인하고, 그렇다면 모델을 `_serving_model_dir` 내 보냅니다.

In [ ]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

`Pusher` 의 출력 결과물을 살펴 보겠습니다. 

In [ ]:
pusher.outputs

특히 Pusher는 다음과 같은 SavedModel 형식으로 모델을 내 보냅니다.

In [ ]:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

내장 TFX 컴포넌트 둘러보기를 마쳤습니다!